In [1]:
import torch
import librosa
import numpy as np
import pandas as pd


In [2]:
def calculate_mfccs(audio_path):
    y, sr = librosa.load(audio_path, sr=48000)
    frame_length = int(0.03 * sr)
    hop_length = int(0.03 * sr)
    
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13, n_fft=frame_length, hop_length=hop_length)
    delta_mfccs = librosa.feature.delta(mfccs)
    delta2_mfccs = librosa.feature.delta(mfccs, order=2)
    return mfccs, delta_mfccs, delta2_mfccs


In [3]:
def extract_spectral_centroid(audio_path):
    y, sr = librosa.load(audio_path, sr=48000)
    frame_length = int(0.03 * sr)
    hop_length = int(0.03 * sr)
    spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr, n_fft=frame_length, hop_length=hop_length)
    return spectral_centroid.flatten()

In [ ]:
def calculate_zcr(audio_path):
    y, sr = librosa.load(audio_path, sr=48000)
    frame_length = int(0.03 * sr)
    hop_length = int(0.03 * sr)
    zcr = librosa.feature.zero_crossing_rate(y, frame_length=frame_length, hop_length=hop_length)
    return zcr.flatten()

In [5]:
import torch
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import yaml
from torch.utils.data import TensorDataset, DataLoader

# class PyTorchModel(nn.Module):
#     def __init__(self, input_size):
#         super(PyTorchModel, self).__init__()
#         # Hidden layers
#         self.layer1 = nn.Sequential(
#             nn.Linear(input_size, 11),
#             nn.ReLU()
#         )
#         self.layer2 = nn.Sequential(
#             nn.Linear(11, 7),
#             nn.ReLU()
#         )
#         self.layer3 = nn.Sequential(
#             nn.Linear(7, 6),
#             nn.ReLU()
#         )
#         # Output layer
#         self.layer4 = nn.Sequential(
#             nn.Linear(6, 4),
#             nn.Softmax(dim=1)
#         )

#     def forward(self, x):
#         x = self.layer1(x)
#         x = self.layer2(x)
#         x = self.layer3(x)
#         x = self.layer4(x)
#         return x
class PyTorchModel(nn.Module):
    def __init__(self, input_size):
        super(PyTorchModel, self).__init__()
        # Define layers with consistent dimensions
        self.layer1 = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.ReLU()
        )
        self.layer2 = nn.Sequential(
            nn.Linear(128, 64),
            nn.ReLU()
        )
        self.layer3 = nn.Sequential(
            nn.Linear(64, 32),
            nn.ReLU()
        )
        # Output layer
        self.layer4 = nn.Sequential(
            nn.Linear(32, 4),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        return x
# class RegularizedClassifier(nn.Module):
#     def __init__(self, input_size=42, num_classes=3):
#         super(RegularizedClassifier, self).__init__()
#         self.model = nn.Sequential(
#             nn.Linear(input_size, 128),
#             nn.ReLU(),
#             nn.BatchNorm1d(128),
#             nn.Dropout(0.4),
#             nn.Linear(128, 256),
#             nn.ReLU(),
#             nn.BatchNorm1d(256),
#             nn.Dropout(0.5),
#             nn.Linear(256, 128),
#             nn.ReLU(),
#             nn.BatchNorm1d(128),
#             nn.Dropout(0.4),
#             nn.Linear(128, 64),
#             nn.ReLU(),
#             nn.BatchNorm1d(64),
#             nn.Dropout(0.3),
#             nn.Linear(64, num_classes),
#             nn.Softmax(dim=1)
#         )

#     def forward(self, x):
#         return self.model(x)

In [6]:
def extract_features(audio_path):
    # Extract all features
    mfccs, delta_mfccs, delta2_mfccs = calculate_mfccs(audio_path)
    spectral_centroid = extract_spectral_centroid(audio_path)
    zcr = calculate_zcr(audio_path)
    
    # Get number of frames (assuming all features have same number of frames)
    num_frames = mfccs.shape[1]
    
    # Initialize list to store features for each frame
    all_features = []
    
    # Combine features for each frame
    for frame in range(num_frames):
        frame_features = []
        # Add MFCC features
        frame_features.extend(mfccs[:, frame])
        frame_features.extend(delta_mfccs[:, frame])
        frame_features.extend(delta2_mfccs[:, frame])
        
        # Add spectral centroid (if frame exists)
        if frame < len(spectral_centroid):
            frame_features.append(spectral_centroid[frame])
            
        # Add ZCR (if frame exists)
        if frame < len(zcr):
            frame_features.append(zcr[frame])
            
        all_features.append(frame_features)
    
    return np.array(all_features)

In [7]:
import torch
import librosa
import numpy as np
import pandas as pd
import os
from torch import nn
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import mode


In [8]:

# def predict_audio(model, audio_path, device='cpu'):
#     model.eval()
    
#     # Extract features
#     features = extract_features(audio_path)
    
#     # Convert to tensor
#     features_tensor = torch.FloatTensor(features).to(device)
    
#     # Make predictions
#     with torch.no_grad():
#         predictions = []
#         for frame_features in features_tensor:
#             output = model(frame_features.unsqueeze(0))
#             pred = output.argmax(dim=1).item()
#             predictions.append(pred)
    
#     # Return mean prediction for the file
#     return np.median(predictions)

def predict_audio(model, audio_path, device='cpu'):
    model.eval()
    
    # Extract features
    features = extract_features(audio_path)
    
    # Convert to tensor
    features_tensor = torch.FloatTensor(features).to(device)
    
    # Make predictions
    with torch.no_grad():
        predictions = []
        for frame_features in features_tensor:
            output = model(frame_features.unsqueeze(0))
            pred = output.argmax(dim=1).item()
            predictions.append(pred)
    
    # Return mode (most common) prediction for the file
    return mode(predictions)[0]

In [9]:
# def evaluate_model(model_path, audio_folder, labels_csv, device='cpu'):
#     # Load the model
#     model = PyTorchModel(input_size=41)  # 41 features: 13 MFCC + 13 delta + 13 delta2 + 1 spectral + 1 ZCR
#     model.load_state_dict(torch.load(model_path, map_location=device))
#     model.to(device)
#     model.eval()
    
#     # Load labels
#     labels_df = pd.read_csv(labels_csv)
    
#     # Lists to store predictions and true labels
#     all_predictions = []
#     all_true_labels = []
    
#     # Dictionary to store file-wise predictions
#     file_predictions = {}
    
#     # Process each audio file
#     for file_name in os.listdir(audio_folder):
#         if file_name.endswith(('.wav', '.ogg')):
#             audio_path = os.path.join(audio_folder, file_name)
            
#             # Extract file number from filename (assuming filename contains the number)
#             try:
#                 file_number = int(''.join(filter(str.isdigit, file_name)))
                
#                 # Get true label from CSV for this file number
#                 true_label = labels_df[labels_df['file_number'] == file_number]['target'].iloc[0]
                
#                 # Make prediction
#                 prediction = predict_audio(model, audio_path, device)
                
#                 # Store results
#                 all_predictions.append(prediction)
#                 all_true_labels.append(true_label)
#                 file_predictions[file_name] = {
#                     'File_Number': file_number,
#                     'True_Label': true_label,
#                     'Predicted_Label': prediction
#                 }
                
#             except (ValueError, IndexError) as e:
#                 print(f"Error processing file {file_name}: {str(e)}")
#                 continue
    
#     # Calculate metrics
#     accuracy = accuracy_score(all_true_labels, all_predictions)
#     class_report = classification_report(all_true_labels, all_predictions)
#     conf_matrix = confusion_matrix(all_true_labels, all_predictions)
    
#     # Create confusion matrix plot
#     plt.figure(figsize=(8, 6))
#     sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
#     plt.title('Confusion Matrix')
#     plt.ylabel('True Label')
#     plt.xlabel('Predicted Label')
#     plt.savefig('confusion_matrix.png')
#     plt.close()
    
#     # Save detailed results to CSV
#     results_df = pd.DataFrame.from_dict(file_predictions, orient='index')
#     results_df.to_csv('detailed_predictions.csv')
    
#     # Print detailed results
#     print(f"\nTotal files processed: {len(file_predictions)}")
#     print(f"Model Accuracy: {accuracy * 100:.2f}%")
#     print("\nClassification Report:")
#     print(class_report)
#     print("\nConfusion Matrix:")
#     print(conf_matrix)
    
#     return accuracy, class_report, conf_matrix, results_df

def evaluate_model(model_path, audio_folder, labels_csv, device='cpu'):
    model = PyTorchModel(input_size=42)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    model.eval()

    labels_df = pd.read_csv(labels_csv)
    all_predictions = []
    all_true_labels = []
    file_predictions = {}

    for file_name in os.listdir(audio_folder):
        if file_name.endswith(('.wav', '.ogg')):
            audio_path = os.path.join(audio_folder, file_name)

            try:
                file_number = int(''.join(filter(str.isdigit, file_name)))
                true_label = labels_df[labels_df['file_number'] == file_number]['target'].iloc[0]
                prediction = predict_audio(model, audio_path, device)

                all_predictions.append(prediction)
                all_true_labels.append(true_label)
                file_predictions[file_name] = {
                    'File_Number': file_number,
                    'True_Label': true_label,
                    'Predicted_Label': prediction
                }

            except (ValueError, IndexError) as e:
                print(f"Error processing file {file_name}: {str(e)}")
                continue

    accuracy = accuracy_score(all_true_labels, all_predictions)
    class_report = classification_report(all_true_labels, all_predictions, target_names=[f'Class {i}' for i in range(1, 4)])
    conf_matrix = confusion_matrix(all_true_labels, all_predictions)

    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=[f'Class {i}' for i in range(1, 4)], yticklabels=[f'Class {i}' for i in range(1, 4)])
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.savefig('confusion_matrix.png')
    plt.close()

    results_df = pd.DataFrame.from_dict(file_predictions, orient='index')
    results_df.to_csv('detailed_predictions.csv')

    print(f"\nTotal files processed: {len(file_predictions)}")
    print(f"Model Accuracy: {accuracy * 100:.2f}%")
    print("\nClassification Report:")
    print(class_report)
    print("\nConfusion Matrix:")
    print(conf_matrix)

    return accuracy, class_report, conf_matrix, results_df


# def evaluate_model(model_path, audio_folder, labels_csv, device='cpu'):
#     # Load the model
#     model = PyTorchModel(input_size=41)  # 41 features: 13 MFCC + 13 delta + 13 delta2 + 1 spectral + 1 ZCR
#     model.load_state_dict(torch.load(model_path, map_location=device))
#     model.to(device)
#     model.eval()
    
#     # Load labels
#     labels_df = pd.read_csv(labels_csv)
    
#     # Lists to store predictions and true labels
#     all_predictions = []
#     all_true_labels = []
    
#     # Dictionary to store file-wise predictions
#     file_predictions = {}
    
#     # Process each audio file
#     for file_name in os.listdir(audio_folder):
#         if file_name.endswith(('.wav', '.ogg')):
#             audio_path = os.path.join(audio_folder, file_name)
            
#             # Extract file number from filename (assuming filename contains the number)
#             try:
#                 file_number = int(''.join(filter(str.isdigit, file_name)))
                
#                 # Get true label from CSV for this file number
#                 true_label = labels_df[labels_df['file_number'] == file_number]['target'].iloc[0]
                
#                 # Make prediction
#                 prediction = predict_audio(model, audio_path, device)
                
#                 # Store results
#                 all_predictions.append(prediction)
#                 all_true_labels.append(true_label)
#                 file_predictions[file_name] = {
#                     'File_Number': file_number,
#                     'True_Label': true_label,
#                     'Predicted_Label': prediction
#                 }
                
#             except (ValueError, IndexError) as e:
#                 print(f"Error processing file {file_name}: {str(e)}")
#                 continue
    
#     # Calculate metrics
#     accuracy = accuracy_score(all_true_labels, all_predictions)
#     class_report = classification_report(all_true_labels, all_predictions)
#     conf_matrix = confusion_matrix(all_true_labels, all_predictions)
    
#     # Create confusion matrix plot
#     plt.figure(figsize=(8, 6))
#     sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
#     plt.title('Confusion Matrix')
#     plt.ylabel('True Label')
#     plt.xlabel('Predicted Label')
#     plt.savefig('confusion_matrix.png')
#     plt.close()
    
#     # Save detailed results to CSV
#     results_df = pd.DataFrame.from_dict(file_predictions, orient='index')
#     results_df.to_csv('detailed_predictions.csv')
    
#     # Print detailed results
#     print(f"\nTotal files processed: {len(file_predictions)}")
#     print(f"Model Accuracy: {accuracy * 100:.2f}%")
#     print("\nClassification Report:")
#     print(class_report)
#     print("\nConfusion Matrix:")
#     print(conf_matrix)
    
#     return accuracy, class_report, conf_matrix, results_df




# def evaluate_model(model_path, audio_folder, labels_csv, device='cpu'):
#     # Load the model
#     model = PyTorchModel(input_size=41)  # 41 features: 13 MFCC + 13 delta + 13 delta2 + 1 spectral + 1 ZCR
#     model.load_state_dict(torch.load(model_path, map_location=device))
#     model.to(device)
#     model.eval()
    
#     # Load labels
#     labels_df = pd.read_csv(labels_csv)
    
#     # Lists to store predictions and true labels
#     all_predictions = []
#     all_true_labels = []
    
#     # Dictionary to store file-wise predictions
#     file_predictions = {}
    
#     # Process each audio file
#     for file_name in os.listdir(audio_folder):
#         if file_name.endswith(('.wav', '.ogg')):
#             audio_path = os.path.join(audio_folder, file_name)
            
#             # Extract file number from filename (assuming filename contains the number)
#             try:
#                 file_number = int(''.join(filter(str.isdigit, file_name)))
                
#                 # Get true label from CSV for this file number
#                 true_label = labels_df[labels_df['file_number'] == file_number]['target'].iloc[0]
                
#                 # Make prediction
#                 prediction = predict_audio(model, audio_path, device)
                
#                 # Convert prediction to class label
#                 if prediction <= 1.5:
#                     predicted_label=1
#                 elif 1.5<prediction<=2:
#                     predicted_label=2
#                 elif 2< prediction<=2.5 :
#                     predicted_label=2
#                 elif 2.5< prediction<=3 :
#                     predicted_label=3
#                 else:
#                     predicted_label=3

                
#                 # Store results
#                 all_predictions.append(predicted_label)
#                 all_true_labels.append(true_label)
#                 file_predictions[file_name] = {
#                     'File_Number': file_number,
#                     'True_Label': true_label,
#                     'Predicted_Label': predicted_label
#                 }
                
#             except (ValueError, IndexError) as e:
#                 print(f"Error processing file {file_name}: {str(e)}")
#                 continue
    
#     # Calculate metrics
#     accuracy = accuracy_score(all_true_labels, all_predictions)
#     class_report = classification_report(all_true_labels, all_predictions)
#     conf_matrix = confusion_matrix(all_true_labels, all_predictions)
    
#     # Create confusion matrix plot
#     plt.figure(figsize=(8, 6))
#     sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
#     plt.title('Confusion Matrix')
#     plt.ylabel('True Label')
#     plt.xlabel('Predicted Label')
#     plt.savefig('confusion_matrix.png')
#     plt.close()
    
#     # Save detailed results to CSV
#     results_df = pd.DataFrame.from_dict(file_predictions, orient='index')
#     results_df.to_csv('detailed_predictions.csv')
    
#     # Print detailed results
#     print(f"\nTotal files processed: {len(file_predictions)}")
#     print(f"Model Accuracy: {accuracy * 100:.2f}%")
#     print("\nClassification Report:")
#     print(class_report)
#     print("\nConfusion Matrix:")
#     print(conf_matrix)
    
#     return accuracy, class_report, conf_matrix, results_df

In [11]:

if __name__ == "__main__":
    # Set your paths here
    model_path = r"C:\Users\OM\Desktop\Anvesshan\codes\model1.pth"
    audio_folder = r"C:\Users\OM\Desktop\Anvesshan\Final_supported_file1"
    labels_csv = r"C:\Users\OM\Desktop\Anvesshan\CSV_files\new_csv_modeified\labeled.csv"
    
    # Run evaluation
    accuracy, class_report, conf_matrix, results_df = evaluate_model(model_path, audio_folder, labels_csv)
    
    # Print results
    print(f"\nModel Accuracy: {accuracy * 100:.2f}%")
    print("\nClassification Report:")
    print(class_report)
    print("\nConfusion Matrix:")
    print(conf_matrix)
    
    # Print detailed file-wise results
    print("\nDetailed predictions have been saved to 'detailed_predictions.csv'")
    print("Confusion matrix plot has been saved as 'confusion_matrix.png'")

: 

In [ ]:
# if __name__ == "__main__":
#     model_path = r"C:\Users\OM\Desktop\Anvesshan\codes\model.pth"
#     audio_path = r"C:\Users\OM\Desktop\Anvesshan\Final_supported_file1\output4.wav"
    
#     predictions = predict_audio(model_path, audio_path)
    
#     # Calculate overall prediction (e.g., majority voting)
#     final_prediction = max(set(predictions), key=predictions.count)
    
#     print(f"Frame-wise predictions: {predictions}")
#     print(f"Final prediction: {final_prediction}")
    
#     # If you want to save predictions to CSV
#     results_df = pd.DataFrame({
#         'Frame': range(len(predictions)),
#         'Prediction': predictions
#     })
#     results_df.to_csv('predictions.csv', index=False)

Frame-wise predictions: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [ ]:
# import torch
# import pandas as pd
# import numpy as np
# import torch
# import torch.nn as nn
# import torch.optim as optim
# import yaml
# from torch.utils.data import TensorDataset, DataLoader

# class PyTorchModel(nn.Module):
#     def __init__(self, input_size):
#         super(PyTorchModel, self).__init__()
#         # Hidden layers
#         self.layer1 = nn.Sequential(
#             nn.Linear(input_size, 11),
#             nn.ReLU()
#         )
#         self.layer2 = nn.Sequential(
#             nn.Linear(11, 7),
#             nn.ReLU()
#         )
#         self.layer3 = nn.Sequential(
#             nn.Linear(7, 6),
#             nn.ReLU()
#         )
#         # Output layer
#         self.layer4 = nn.Sequential(
#             nn.Linear(6, 4),
#             nn.Softmax(dim=1)
#         )

#     def forward(self, x):
#         x = self.layer1(x)
#         x = self.layer2(x)
#         x = self.layer3(x)
#         x = self.layer4(x)
#         return x
# input_size=41
# model = PyTorchModel(input_size)

# model.load_state_dict(torch.load(r"C:\Users\OM\Desktop\Anvesshan\codes\model.pth"))

# model.eval()

PyTorchModel(
  (layer1): Sequential(
    (0): Linear(in_features=41, out_features=11, bias=True)
    (1): ReLU()
  )
  (layer2): Sequential(
    (0): Linear(in_features=11, out_features=7, bias=True)
    (1): ReLU()
  )
  (layer3): Sequential(
    (0): Linear(in_features=7, out_features=6, bias=True)
    (1): ReLU()
  )
  (layer4): Sequential(
    (0): Linear(in_features=6, out_features=4, bias=True)
    (1): Softmax(dim=1)
  )
)

In [50]:
# audio_file_path = r"C:\Users\OM\Desktop\Anvesshan\Final_supported_file1\output11.wav"
# severity_prediction = preprocess_and_predict(audio_file_path, model)
# print(f"The severity level for the audio file is: {severity_prediction}")

In [4]:
import numpy as np 
import pandas as pd 

In [5]:
df=pd.read_csv(r"C:\Users\OM\Desktop\Anvesshan\CSV_files\new_csv_modeified\new_features_balanced_withlabels.csv")

In [6]:
df

,Unnamed: 0,MFCC_1,Delta_MFCC_1,Delta2_MFCC_1,MFCC_2,Delta_MFCC_2,Delta2_MFCC_2,MFCC_3,Delta_MFCC_3,Delta2_MFCC_3,...,MFCC_12,Delta_MFCC_12,Delta2_MFCC_12,MFCC_13,Delta_MFCC_13,Delta2_MFCC_13,Spectral Centroid,ZCR,file_number,target
0,0,-617.136350,-21.809950,-0.982294,0.000000,-10.088035,-5.530091,0.000000,-0.422524,-4.340887,...,0.000000,-0.781564,-0.276063,0.000000,-0.603166,-0.109370,4139.468051,0.095139,1,2
1,1,-245.365920,-21.809950,-0.982294,134.041760,-10.088035,-5.530091,-18.568258,-0.422524,-4.340887,...,10.693646,-0.781564,-0.276063,8.920538,-0.603166,-0.109370,3092.952786,0.050000,1,2
2,2,-520.232900,-21.809950,-0.982294,108.452774,-10.088035,-5.530091,58.410040,-0.422524,-4.340887,...,11.534636,-0.781564,-0.276063,10.259850,-0.603166,-0.109370,996.898621,0.004167,1,2
3,3,-559.519200,-21.809950,-0.982294,63.572136,-10.088035,-5.530091,29.783543,-0.422524,-4.340887,...,4.968514,-0.781564,-0.276063,-0.408971,-0.603166,-0.109370,1941.750887,0.025000,1,2
4,4,-567.658900,-21.809950,-0.982294,58.891804,-10.088035,-5.530091,35.398216,-0.422524,-4.340887,...,-2.926584,-0.781564,-0.276063,-5.431875,-0.603166,-0.109370,1127.026406,0.021528,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1223371,1223371,-535.426505,59.283352,19.468298,10.679212,18.810494,6.399599,0.915076,-16.400836,-4.313193,...,-4.855188,4.034524,0.955544,-6.020536,3.800120,0.514288,2835.969717,0.074670,1216,3
1223372,1223372,-619.857360,0.014272,-0.017408,0.000000,0.018298,-0.021403,0.000000,0.016005,-0.018183,...,0.000000,-0.015041,0.015043,0.000000,-0.017892,0.018761,4628.435903,0.164807,1540,3
1223373,1223373,-158.871714,55.605779,-22.130607,229.550045,30.662307,-12.857187,-70.149186,-13.942554,1.651155,...,-11.111302,-0.094466,0.419982,-17.727523,-1.952745,0.515949,1924.977599,0.040841,823,3
1223374,1223374,-540.500530,-0.774177,0.329081,0.506638,-0.609312,0.209496,0.203382,0.153853,-0.141647,...,0.237027,0.047260,-0.025773,0.153589,-0.071259,0.006203,5596.928857,0.146685,1542,3


In [7]:
df = df.sort_values(by='file_number').reset_index(drop=True)

In [8]:
df

,Unnamed: 0,MFCC_1,Delta_MFCC_1,Delta2_MFCC_1,MFCC_2,Delta_MFCC_2,Delta2_MFCC_2,MFCC_3,Delta_MFCC_3,Delta2_MFCC_3,...,MFCC_12,Delta_MFCC_12,Delta2_MFCC_12,MFCC_13,Delta_MFCC_13,Delta2_MFCC_13,Spectral Centroid,ZCR,file_number,target
0,0,-617.13635,-2.180995e+01,-9.822940e-01,0.000000,-10.088035,-5.530091,0.000000,-0.422524,-4.340887,...,0.000000,-0.781564,-0.276063,0.000000,-0.603166,-0.109370,4139.468051,0.095139,1,2
1,224,-614.78314,-2.545899e-01,-1.158011e+00,3.305869,-0.321633,-1.546758,3.240516,-0.247597,-1.365138,...,1.345665,0.036983,-0.151246,1.088671,0.079846,0.033752,2759.535940,0.035417,1,2
2,223,-589.30347,3.402496e-01,-1.124637e+00,36.922974,0.477637,-1.485977,32.082195,0.467088,-1.279622,...,1.959804,0.125394,-0.087983,-2.655253,0.072443,0.093085,633.665363,0.021528,1,2
3,222,-617.13635,7.223663e-01,-9.316410e-01,0.000000,0.978398,-1.228830,0.000000,0.890193,-1.053752,...,0.000000,0.148190,-0.049562,0.000000,0.034617,0.103188,3761.232254,0.058333,1,2
4,221,-617.13635,1.112338e+00,-3.854683e-01,0.000000,1.490185,-0.503701,0.000000,1.324052,-0.422237,...,0.000000,0.173076,0.023240,0.000000,-0.002449,0.091224,4092.381643,0.065972,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1223371,605584,-538.22860,1.867356e-15,-7.105427e-15,0.000000,-0.000000,0.000000,0.000000,-0.000000,0.000000,...,0.000000,-0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000,0.000000,2165,1
1223372,605583,-538.22860,1.867356e-15,-7.105427e-15,0.000000,-0.000000,0.000000,0.000000,-0.000000,0.000000,...,0.000000,-0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000,0.000000,2165,1
1223373,605582,-538.22860,1.867356e-15,-7.105427e-15,0.000000,-0.000000,0.000000,0.000000,-0.000000,0.000000,...,0.000000,-0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000,0.000000,2165,1
1223374,605581,-538.22860,1.867356e-15,-7.105427e-15,0.000000,-0.000000,0.000000,0.000000,-0.000000,0.000000,...,0.000000,-0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000,0.000000,2165,1


In [10]:
df = df.drop(
    columns=[
    "Unnamed: 0","MFCC_1","Delta_MFCC_1","Delta2_MFCC_1","MFCC_2","Delta_MFCC_2","Delta2_MFCC_2","MFCC_3","Delta_MFCC_3","Delta2_MFCC_3","Delta_MFCC_12	"
    ,"Delta2_MFCC_13","Spectral Centroid","File Name","ZCR"
    
    ], 
    axis=1, 
    errors='ignore'
)

In [11]:
df

,MFCC_4,Delta_MFCC_4,Delta2_MFCC_4,MFCC_5,Delta_MFCC_5,Delta2_MFCC_5,MFCC_6,Delta_MFCC_6,Delta2_MFCC_6,MFCC_7,...,MFCC_11,Delta_MFCC_11,Delta2_MFCC_11,MFCC_12,Delta_MFCC_12,Delta2_MFCC_12,MFCC_13,Delta_MFCC_13,file_number,target
0,0.000000,-1.534546,-1.527144,0.000000,-0.820079,-1.413307,0.000000,-1.663707,-1.264868,0.000000,...,0.000000,-0.628276,-0.702974,0.000000,-0.781564,-0.276063,0.000000,-0.603166,1,2
1,3.133850,-0.208791,-1.249576,2.989125,-0.217285,-1.227291,2.810664,-0.216793,-1.174783,2.603631,...,1.607562,0.024944,-0.266827,1.345665,0.036983,-0.151246,1.088671,0.079846,1,2
2,29.097107,0.449722,-1.155706,28.703316,0.425862,-1.146027,27.539013,0.395945,-1.104983,22.710442,...,4.687932,0.177500,-0.187953,1.959804,0.125394,-0.087983,-2.655253,0.072443,1,2
3,0.000000,0.828069,-0.948948,0.000000,0.804540,-0.941423,0.000000,0.763917,-0.907391,0.000000,...,0.000000,0.229834,-0.133379,0.000000,0.148190,-0.049562,0.000000,0.034617,1,2
4,0.000000,1.216727,-0.374426,0.000000,1.192920,-0.372931,0.000000,1.140828,-0.359437,0.000000,...,0.000000,0.285568,-0.012342,0.000000,0.173076,0.023240,0.000000,-0.002449,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1223371,0.000000,-0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000,...,0.000000,-0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000,-0.000000,2165,1
1223372,0.000000,-0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000,...,0.000000,-0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000,-0.000000,2165,1
1223373,0.000000,-0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000,...,0.000000,-0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000,-0.000000,2165,1
1223374,0.000000,-0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000,...,0.000000,-0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000,-0.000000,2165,1


In [12]:
df = df.drop(
    columns=[
    "MFCC_4","Delta_MFCC_4","Delta2_MFCC_4","MFCC_5","Delta_MFCC_5","Delta2_MFCC_5","MFCC_6",
    "Delta_MFCC_6","Delta2_MFCC_6","MFCC_7","Delta_MFCC_7","Delta2_MFCC_7","MFCC_8","Delta_MFCC_8","Delta2_MFCC_8","MFCC_9",
    "Delta_MFCC_9","Delta2_MFCC_9","MFCC_10","Delta_MFCC_10","Delta2_MFCC_10","MFCC_11","Delta_MFCC_11","Delta2_MFCC_11","MFCC_12","Delta_MFCC_12",
   "Delta2_MFCC_12","MFCC_13","Delta_MFCC_13"
    
    ], 
    axis=1, 
    errors='ignore'
)

In [13]:
df

,file_number,target
0,1,2
1,1,2
2,1,2
3,1,2
4,1,2
...,...,...
1223371,2165,1
1223372,2165,1
1223373,2165,1
1223374,2165,1


In [14]:
df=df.to_csv(r"C:\Users\OM\Desktop\Anvesshan\CSV_files\new_csv_modeified\labeled.csv")

In [77]:
# Load true labels
labels_df = pd.read_csv(r"C:\Users\OM\Desktop\Anvesshan\CSV_files\new_csv_modeified\labeled.csv")  # Assuming columns: 'File Name' and 'Label'
true_labels = dict(zip(labels_df['file_number'], labels_df['target']))

In [78]:
# Folder containing the audio files
folder_path = r"C:\Users\OM\Desktop\Anvesshan\Final_supported_file1"

In [79]:

import torch
import os
import pandas as pd

correct_predictions=0
total_predictions=0
# Loop through audio files in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith('.wav') or file_name.endswith('.ogg'):
        audio_path = os.path.join(folder_path, file_name)
        
        # Get the model prediction
        predicted_class = preprocess_and_predict(audio_path, model)
        
        # Compare with true label
        if file_name in true_labels:
            true_label = true_labels[file_name]
            if predicted_class == true_label:
                correct_predictions += 1
            total_predictions += 1

In [84]:
accuracy = correct_predictions / total_predictions if total_predictions > 0 else 0
print(f"Model accuracy on provided folder: {accuracy * 100:.2f}%")

Model accuracy on provided folder: 0.00%
